# Data Modeling with Apache Cassandra

## Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

##### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

##### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

## Part II. Apache Cassandra Coding Portion

### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

# APACHE CASSANDRA RPOJECT

Create queries to ask the following three questions of the data:

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Database creation 

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
# Create a Keyspace 

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_database 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [4]:
# Set to the keyspace specified above
session.set_keyspace('sparkify_database')

#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## QUERY 1

**Question 1** : Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

**Tought Process** : 

*See Notebook 'Data_Exploration' to have a more detailed explanation*
- SessionId is a unique variable that groups together a unique session (no duplicate SessionId between users). **Thus: SessionId is a good choice to partition our data.** 
- Each session contains the songs listened during this specific session. Each song has a unique ID named itemInSession. **Thus: ItemInSession is a good choice to create a unique primary key.** 
    - Info about the song (name, lenght, artist) is unique for the row
    - Info about the user (userId, Name, Level) is unique to the session

Chosen Primary key: 
- Partition Key: sessionId 
- Clustering Key: itemInSession
- Columns of interests: artist, song title and song's length

In [5]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Drop the previous table (to start from scratch - specific to this learning context)
session.execute("""DROP TABLE IF EXISTS sessions_library""")

# Create the new table clustered by sessions
session.execute("""CREATE TABLE IF NOT EXISTS sessions_library (
                                        sessionId       int,
                                        itemInSession   int,
                                        artist_name     text, 
                                        song_title      text, 
                                        song_length     float, 

                                        PRIMARY KEY (sessionId, itemInSession))""") 

In [6]:
# Add data in table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sessions_library (sessionId, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

##### Do a SELECT to verify that the data have been inserted into each table

In [7]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query_Q1 = ("""SELECT * 
               FROM sessions_library 
               WHERE sessionId = 338 AND itemInSession = 4""")

results = pd.DataFrame(list(session.execute(query_Q1)))
results

,sessionid,iteminsession,artist_name,song_length,song_title
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


## QUERY 2

**Question 2** : Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


**Tought Process** : 

*See Notebook 'Data_Exploration' to have a more detailed explanation*

- Our request is to find the user #10 with the sessionId #182. Our first partition key should be the userId since it's unique to the user. This user has many session (each containing more informations about songs played). It is better to further partition our data by sessionId. **Thus: we use 1) userId as first partition key and 2) sessionId as second partition key**
- We also need a unique id to keep all the songs played in a unique session (otherwise only the last one added will be kept). As in query 1: ItemInSession is perfect for clustering and allows to sort by it. **Thus: itemInSession is used to created a unique song ID for the session and allow to sort by itemInSession**

Chosen Primary key: 
- Partition Key: userId, sessionId
- Clustering Key: itemInSession
- Columns of interests: artist_name, song, user name (first and last)

In [20]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) where sessionId = 338, and itemInSession = 4

# Drop the previous table (to start from scratch - specific to this learning context)
session.execute("""DROP TABLE IF EXISTS users_library""")

# Create the new table clustered by sessions
session.execute("""CREATE TABLE IF NOT EXISTS users_library (
                                        userId          int,
                                        sessionId       int,
                                        itemInSession   int,
                                        artist_name     text, 
                                        song_title      text, 
                                        user_firstName  text,
                                        user_lastName   text, 

                                        PRIMARY KEY ((userId, sessionId), itemInSession))""") 

In [21]:
# Add data in table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_library (userId, sessionId, itemInSession, artist_name, song_title, user_firstName, user_lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [22]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) where sessionId = 338, and itemInSession = 4

query_Q2 = ("""SELECT artist_name, song_title, itemInSession, user_firstName, user_lastName
               FROM users_library 
               WHERE userId = 10 AND sessionId = 182""")

results = pd.DataFrame(list(session.execute(query_Q2)))
results              

,artist_name,song_title,iteminsession,user_firstname,user_lastname
0,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
1,Three Drives,Greece 2000,1,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



#### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

#### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()